# FIT5226 Project: Multi‐Agent Delivery with Q‐Learning  
 
**Overview:**  
In this project we implement a tabular Q‐learning solution for a two‐agent pickup‐and‐dropoff task on a 5×5 grid. Agents incur a penalty for head‐on collisions and receive a reward for successful deliveries. We will:

1. Define a **GridWorld** environment class.  
2. Encapsulate reward logic in a **RewardStructure** class.  
3. Implement a **QLearningAgent** with ε‐greedy policy and decay.  
4. Train with **early stopping** once a target success rate is reached within 1,000,000 agent‐steps.  
5. Compare two ε‐decay schedules and visualise results.  
6. Fully document design choices, results, and limitations.


In [24]:
import os
import numpy as np
import random
import pickle
import time

# === Hyperparameters ===
GRID_SIZE = 5
NUM_AGENTS = 4
ALPHA = 0.005         # learning rate
# ALPHA = 1e-3            # learning rate
GAMMA = 0.95           # discount factor
EPSILON_START = 1.0    # initial exploration rate
EPSILON_DECAY = 0.9997  # Try a slightly faster decay
EPSILON_MIN = 0.05      # Allow more exploitation
MAX_EPISODE_STEPS = 40 # match evaluation horizon
STEP_BUDGET = 1_500_000  # total agent-steps budget

# === Reward weights ===
REWARD_STEP      = -0.01  # per-step penalty
REWARD_PICKUP    = +2.0  # pickup reward
REWARD_DELIVERY = +10.0 # delivery reward (amplified)
REWARD_COLLISION = -25.0 # collision penalty
SHAPING_COEFF    = 0.2   # potential-based shaping coefficient

# === Training flags ===
USE_SENSOR = True      # disable sensor to reduce state-space initially
USE_OFFJOB_TRAINING = True
USE_CENTRAL_CLOCK = True

# Precompute all (A,B) pairs for curriculum sampling
ALL_PAIRS = [((x, y), (u, v))
             for x in range(GRID_SIZE) for y in range(GRID_SIZE)
             for u in range(GRID_SIZE) for v in range(GRID_SIZE)
             if (x, y) != (u, v)]

coverage = {pair: 0 for pair in ALL_PAIRS}

def randomize_locations():
    """
    Sample (A,B) inversely proportional to coverage for curriculum.
    """
    weights = [1.0 / (coverage[pair] + 1)**2 for pair in ALL_PAIRS]
    pair = random.choices(ALL_PAIRS, weights=weights, k=1)[0]
    coverage[pair] += 1
    return pair


In [25]:
class Agent:
    def __init__(self, agent_id, shared_q):
        self.id = agent_id
        self.q_table = shared_q
        self.epsilon = EPSILON_START
        self.reset(None, False)

    def reset(self, start_pos, carrying):
        self.pos = start_pos
        self.carrying = carrying
        self.last_state = None
        self.last_action = None

    def get_state(self, grid, A_loc, B_loc):
        x, y = self.pos
        c = int(self.carrying)
        mask = 0
        if USE_SENSOR:
            dirs = [(-1,-1),(-1,0),(-1,1),(0,-1),(0,1),(1,-1),(1,0),(1,1)]
            for i, (dx,dy) in enumerate(dirs):
                nx, ny = x+dx, y+dy
                if 0 <= nx < GRID_SIZE and 0 <= ny < GRID_SIZE:
                    cell = grid[nx][ny]
                    if cell:
                        for occ_id, occ_carry in cell:
                            if occ_id != self.id and occ_carry != c:
                                mask |= (1 << i)
                                break
        dxA, dyA = A_loc[0] - x, A_loc[1] - y
        dxB, dyB = B_loc[0] - x, B_loc[1] - y
        return (x, y, c, mask, dxA, dyA, dxB, dyB)

    def choose_action(self, state):
        if random.random() < self.epsilon:
            return random.randrange(4)
        q = self.q_table.setdefault(state, np.zeros(4))
        return int(np.argmax(q))

    def update_q(self, state, action, reward, next_state):
        q = self.q_table.setdefault(state, np.zeros(4))
        q_next = self.q_table.setdefault(next_state, np.zeros(4))
        q[action] += ALPHA * (reward + GAMMA * np.max(q_next) - q[action])


def step_all_agents(agents, grid, A_loc, B_loc, train=True):
    proposals = []
    for agent in agents:
        # record old distance for shaping
        if agent.carrying:
            old_dist = abs(agent.pos[0]-B_loc[0]) + abs(agent.pos[1]-B_loc[1])
        else:
            old_dist = abs(agent.pos[0]-A_loc[0]) + abs(agent.pos[1]-A_loc[1])
        state = agent.get_state(grid, A_loc, B_loc)
        action = agent.choose_action(state)
        dx, dy = [(-1,0),(1,0),(0,-1),(0,1)][action]
        nx, ny = agent.pos[0] + dx, agent.pos[1] + dy
        if not (0 <= nx < GRID_SIZE and 0 <= ny < GRID_SIZE):
            nx, ny = agent.pos
        proposals.append((agent, agent.pos, (nx, ny), action, state, old_dist, agent.carrying))

    # detect head-on collisions
    collisions = set()
    for i, (ai, old_i, new_i, _, _, _, _) in enumerate(proposals):
        for j, (aj, old_j, new_j, _, _, _, _) in enumerate(proposals):
            if i < j and old_i == new_j and old_j == new_i and ai.carrying != aj.carrying:
                if old_i not in [A_loc, B_loc] and old_j not in [A_loc, B_loc]:
                    collisions.add((ai.id, aj.id))

    new_grid = [[[] for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
    total_reward = 0.0
    for agent, old_pos, new_pos, action, state, old_dist, carried_before in proposals:
        # per-step + shaping
        reward = REWARD_STEP
        if carried_before:
            new_dist = abs(new_pos[0]-B_loc[0]) + abs(new_pos[1]-B_loc[1])
        else:
            new_dist = abs(new_pos[0]-A_loc[0]) + abs(new_pos[1]-A_loc[1])
        reward += SHAPING_COEFF * (old_dist - new_dist)
        # collision penalty
        if any(agent.id in pair for pair in collisions):
            reward += REWARD_COLLISION
        agent.pos = new_pos
        # pickup/delivery
        if not agent.carrying and agent.pos == A_loc:
            agent.carrying = True
            reward += REWARD_PICKUP
        elif agent.carrying and agent.pos == B_loc:
            agent.carrying = False
            reward += REWARD_DELIVERY
        new_grid[agent.pos[0]][agent.pos[1]].append((agent.id, agent.carrying))
        next_state = agent.get_state(new_grid, A_loc, B_loc)
        if train:
            agent.update_q(state, action, reward, next_state)
        total_reward += reward
    return new_grid, len(collisions), total_reward

In [26]:
def train(num_episodes=10000, collision_budget=3998, time_budget=600):
    shared_q = {}
    agents = [Agent(i, shared_q) for i in range(NUM_AGENTS)]
    total_collisions = 0
    total_steps      = 0
    start_time = time.time()
    coverage = {}

    for ep in range(1, num_episodes+1):
        ep_start_time = time.time()
        ep_collisions = 0
        ep_steps      = 0

        # 1) sample (A,B) and track coverage
        A_loc, B_loc = randomize_locations()
        coverage[(A_loc, B_loc)] = coverage.get((A_loc, B_loc), 0) + 1

        # 2) reset grid
        grid = [[[] for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]

        # 3) pick start positions
        if USE_OFFJOB_TRAINING:
            starts = [random.choice([A_loc, B_loc]) for _ in range(NUM_AGENTS)]
        else:
            starts = [A_loc] * (NUM_AGENTS//2) + [B_loc] * (NUM_AGENTS - NUM_AGENTS//2)
            random.shuffle(starts)

        # 4) reset each agent so that agent.pos is never None
        for agent, start in zip(agents, starts):
            initial_carry = (start == A_loc)
            agent.reset(start, initial_carry)
            grid[start[0]][start[1]].append((agent.id, agent.carrying))

        # --- LOGGING: Print episode and agent starts like evaluation ---
        print(f"\n[Train] Episode {ep}: A={A_loc}, B={B_loc}")
        for agent, start in zip(agents, starts):
            loc_str = "A_loc" if start == A_loc else "B_loc"
            print(f"    Agent {agent.id} starts at {loc_str} ({start})")
        # -------------------------------------------------------------

        # --- Advanced per-agent cycle step tracking (A→B→A or B→A→B) ---
        cycle_start = {ag.id: None for ag in agents}
        visited_other = {ag.id: False for ag in agents}
        completion_step = {ag.id: None for ag in agents}

        for step in range(1, MAX_EPISODE_STEPS + 1):
            ep_steps += 1
            order = agents if USE_CENTRAL_CLOCK else random.sample(agents, len(agents))
            grid, collisions, *_ = step_all_agents(order, grid, A_loc, B_loc)
            ep_collisions += collisions
            total_collisions += collisions  # <-- ADD THIS LINE

            for ag, start in zip(agents, starts):
                # Mark the start of the cycle
                if cycle_start[ag.id] is None:
                    cycle_start[ag.id] = step
                # Determine the "other" location
                other_loc = B_loc if start == A_loc else A_loc
                # Mark if agent has visited the other location
                if not visited_other[ag.id] and ag.pos == other_loc:
                    visited_other[ag.id] = True
                # If agent has returned to start after visiting other, cycle is complete
                if (visited_other[ag.id] and ag.pos == start and
                    completion_step[ag.id] is None and step > cycle_start[ag.id]):
                    completion_step[ag.id] = step - cycle_start[ag.id] + 1

            # If all agents have completed a cycle, stop early
            if all(completion_step[ag.id] is not None for ag in agents):
                break
        # -------------------------------------------------------------

        # Log per-agent cycle steps
        for ag in agents:
            s = completion_step[ag.id]
            if s is not None and s <= MAX_EPISODE_STEPS:
                print(f"  Agent {ag.id} completed cycle in {s} steps")
            elif s is not None:
                print(f"  Agent {ag.id} completed cycle in {s} steps (exceeds {MAX_EPISODE_STEPS})")
            else:
                print(f"  Agent {ag.id} did not complete a full cycle in {MAX_EPISODE_STEPS} steps")

        total_steps += ep_steps

        # 6) decay epsilon
        for agent in agents:
            agent.epsilon = max(EPSILON_MIN, agent.epsilon * EPSILON_DECAY)

        # 7) log this episode
        ep_time = time.time() - ep_start_time
        print(f"[Train] Ep {ep:5d} → steps: {ep_steps:3d}, "
              f"collisions: {ep_collisions:2d}, time: {ep_time:5.2f}s")

        # 8) optional exploration burst
        if ep % 10000 == 0 and ep <= num_episodes - 10000:
            for agent in agents:
                agent.epsilon = EPSILON_START
            print(f"*** Exploration burst at ep {ep} ***")

        if total_collisions > collision_budget:
            print(f"Collision budget exceeded at episode {ep}. Stopping training.")
            break
        if (time.time() - start_time) > time_budget:
            print(f"Time budget exceeded at episode {ep}. Stopping training.")
            break

    # end episodes loop

    # report least-trained pairs
    least = sorted(coverage.items(), key=lambda kv: kv[1])[:5]
    print("Least-trained (A,B) pairs and counts:", least)

    # save Q-table
    with open("q_table.pkl", "wb") as f:
        pickle.dump(shared_q, f)

    # final summary
    total_time = time.time() - start_time
    print(f"\nTraining complete.")
    print(f"  Total episodes run : {ep}")
    print(f"  Total agent-steps  : {total_steps}")
    print(f"  Total collisions   : {total_collisions}")
    print(f"  Total wall-time    : {total_time:.2f} seconds")

    return shared_q

q_table = train()


[Train] Episode 1: A=(4, 2), B=(1, 1)
    Agent 0 starts at B_loc ((1, 1))
    Agent 1 starts at B_loc ((1, 1))
    Agent 2 starts at B_loc ((1, 1))
    Agent 3 starts at B_loc ((1, 1))
  Agent 0 did not complete a full cycle in 40 steps
  Agent 1 did not complete a full cycle in 40 steps
  Agent 2 did not complete a full cycle in 40 steps
  Agent 3 did not complete a full cycle in 40 steps
[Train] Ep     1 → steps:  40, collisions:  0, time:  0.00s

[Train] Episode 2: A=(1, 1), B=(4, 4)
    Agent 0 starts at B_loc ((4, 4))
    Agent 1 starts at A_loc ((1, 1))
    Agent 2 starts at A_loc ((1, 1))
    Agent 3 starts at A_loc ((1, 1))
  Agent 0 did not complete a full cycle in 40 steps
  Agent 1 did not complete a full cycle in 40 steps
  Agent 2 did not complete a full cycle in 40 steps
  Agent 3 did not complete a full cycle in 40 steps
[Train] Ep     2 → steps:  40, collisions:  2, time:  0.00s

[Train] Episode 3: A=(1, 1), B=(1, 0)
    Agent 0 starts at A_loc ((1, 1))
    Agent 1 st

In [27]:

def all_agent_distributions(A_loc, B_loc, num_agents=4):
    # Only include distributions with at least 1 agent at B (so: 1B3A, 2B2A, 3B1A, 4B0A)
    distributions = []
    for b_count in range(1, num_agents + 1):  # b_count: 1,2,3,4
        a_count = num_agents - b_count
        starts = [B_loc] * b_count + [A_loc] * a_count
        distributions.append(starts)
    return distributions

def evaluate(q_table, scenarios=None, max_steps=25):
    if scenarios is None:
        scenarios = sorted(set(ALL_PAIRS))  # Ensure no duplicates
    total = len(scenarios) * 4  # 4 distributions per scenario (1B3A, 2B2A, 3B1A, 4B0A)
    successes = 0
    print(f"Number of ALL_PAIRS: {len(scenarios)}")
    print(f"Total expected episodes: {total}")
    print("Starting evaluation with all agent distributions (at least 1 at B)...")

    idx = 0
    for (A_loc, B_loc) in scenarios:
        for starts in all_agent_distributions(A_loc, B_loc, NUM_AGENTS):
            idx += 1
            print(f"\nEpisode {idx}/{total}: A={A_loc}, B={B_loc}, starts={starts}")
            grid = [[[] for _ in range(GRID_SIZE)] for _ in range(GRID_SIZE)]
            agents = [Agent(i, q_table) for i in range(NUM_AGENTS)]
            for agent, start in zip(agents, starts):
                initial_carry = (start == A_loc)
                agent.reset(start, initial_carry)
                agent.epsilon = 0.0
                grid[start[0]][start[1]].append((agent.id, agent.carrying))

            picked_up = {ag.id: False for ag in agents}
            cycle_start = {ag.id: None for ag in agents}
            completion_step = {ag.id: None for ag in agents}
            collision_at = None

            for step in range(1, max_steps * 2 + 1):
                grid, collisions, _ = step_all_agents(agents, grid, A_loc, B_loc, train=False)
                if collisions > 0:
                    collision_at = step
                    break
                for ag, start in zip(agents, starts):
                    if start == A_loc and cycle_start[ag.id] is None and ag.pos == B_loc:
                        cycle_start[ag.id] = step
                        picked_up[ag.id] = False
                    if start == B_loc:
                        cycle_start[ag.id] = 1
                    if cycle_start[ag.id] is not None and step >= cycle_start[ag.id]:
                        if not picked_up[ag.id] and ag.pos == A_loc and ag.carrying:
                            picked_up[ag.id] = True
                        if picked_up[ag.id] and not ag.carrying and ag.pos == B_loc and completion_step[ag.id] is None:
                            completion_step[ag.id] = step - cycle_start[ag.id] + 1
                if all(completion_step[ag.id] is not None for ag in agents):
                    break

            for ag in agents:
                s = completion_step[ag.id]
                if collision_at is not None:
                    print(f"  Agent {ag.id} did not finish due to collision at step {collision_at}")
                elif s is not None and s <= max_steps:
                    print(f"  Agent {ag.id} succeeded in B→A→B cycle in {s} steps")
                elif s is not None:
                    print(f"  Agent {ag.id} completed cycle in {s} steps (exceeds {max_steps})")
                else:
                    print(f"  Agent {ag.id} failed to complete B→A→B cycle within {max_steps} steps")

            episode_success = (
                collision_at is None and
                all(completion_step[ag.id] is not None and completion_step[ag.id] <= max_steps for ag in agents)
            )
            status = "SUCCESS" if episode_success else "FAILURE"
            print(f"Episode {idx} Result: {status}")
            if episode_success:
                successes += 1

    print(f"\nEpisodes actually run: {idx}")
    rate = 100 * successes / total
    print(f"\nOverall success: {successes}/{total} episodes → {rate:.2f}%")
    return rate


In [28]:
# Load existing Q-table or train a new one
if os.path.exists("q_table.pkl"):
    with open("q_table.pkl", "rb") as f:
        q_table = pickle.load(f)
    print("Loaded existing Q-table for evaluation.")
else:
    print("No Q-table found; starting training...")
    q_table = train()
# Evaluate the policy
final_rate = evaluate(q_table)
print(f"Overall success rate: {final_rate:.2f}%")

Loaded existing Q-table for evaluation.
Number of ALL_PAIRS: 600
Total expected episodes: 2400
Starting evaluation with all agent distributions (at least 1 at B)...

Episode 1/2400: A=(0, 0), B=(0, 1), starts=[(0, 1), (0, 0), (0, 0), (0, 0)]
  Agent 0 succeeded in B→A→B cycle in 2 steps
  Agent 1 succeeded in B→A→B cycle in 3 steps
  Agent 2 succeeded in B→A→B cycle in 3 steps
  Agent 3 succeeded in B→A→B cycle in 3 steps
Episode 1 Result: SUCCESS

Episode 2/2400: A=(0, 0), B=(0, 1), starts=[(0, 1), (0, 1), (0, 0), (0, 0)]
  Agent 0 succeeded in B→A→B cycle in 2 steps
  Agent 1 succeeded in B→A→B cycle in 2 steps
  Agent 2 succeeded in B→A→B cycle in 3 steps
  Agent 3 succeeded in B→A→B cycle in 3 steps
Episode 2 Result: SUCCESS

Episode 3/2400: A=(0, 0), B=(0, 1), starts=[(0, 1), (0, 1), (0, 1), (0, 0)]
  Agent 0 succeeded in B→A→B cycle in 2 steps
  Agent 1 succeeded in B→A→B cycle in 2 steps
  Agent 2 succeeded in B→A→B cycle in 2 steps
  Agent 3 succeeded in B→A→B cycle in 3 steps
